**Задание по проекту.** 
Для его выполнения вам понадобится собранная коллекция документов и функция, составляющая обратный индекс по словам в коллекции.

Напишите функцию (или несколько отдельных логичный функций), которая по запросу $Q = q_1,..., g_n$ и коллекции $D$ сортирует выдачу подходящих документов. Будем считать документ подходящим, если он содержит хотя бы одно слово из запроса (из которого удалены стоп-слова). В качестве метрики используйте *Okapi BM25*.

Для проверки работы функции на вашем корпусе используйте запрос **каникулы на новый год и рождество**. Выведите ссылки в ipynb на первые десять докуменов в отсортированной выдаче(как во втором семинаре с помощью IPython.display) и их оценку BM25. Напомню, что ссылки на документы хрянятся в самих доках под тэгом @url.

Про что не забыть:
1. Лемматизируем запрос, удаляем стоп-слова => запрос готов
2. Лемматизируем слова в документах => документы готовы к подсчетам статистик по запросу

In [1]:
import os
import nltk
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
from string import punctuation
from collections import defaultdict
from pymorphy2 import MorphAnalyzer
from math import log
from IPython.display import HTML, display
import json

In [2]:
k1 = 2.0
b = 0.75

def score_BM25(n, fq, N, dl, avdl):
    K = compute_K(dl, avdl)
    IDF = log((N - n + 0.5) / (n + 0.5))
    frac = ((k1 + 1) * fq) / (K + fq)
    return IDF * frac


def compute_K(dl, avdl):
    return k1 * ((1-b) + b * (float(dl)/float(avdl)))

In [3]:
def lemmatisation(text):
    morph = MorphAnalyzer()
    stop_words = stopwords.words('russian')
    exclude = set(punctuation + '0123456789'+'–—'+'«»')
    text = ''.join(ch for ch in text if ch not in exclude)
    tokens = WhitespaceTokenizer().tokenize(text.lower()) 
    tokens = [i.strip() for i in tokens if i not in stop_words]
    lemmas = [morph.parse(i)[0].normal_form for i in tokens]
    return lemmas

In [4]:
def search(query):
    query = lemmatisation(query)
    relevant_urls = defaultdict(float)
    for lemma in query:
        if lemma in inv_index:
            lemma_info = inv_index[lemma]
            n = len(lemma_info)
            for i in lemma_info:
                fq = i[1]
                dl = url_info[i[0]][0]
                link = '<a href="' + i[0] + '">' + url_info[i[0]][1] + '</a>'
                relevant_urls[link] += score_BM25(n, fq, N, dl, avgdl)
    res = sorted(relevant_urls.items(),key=lambda x: x[1],reverse=True)
    return res

In [5]:
with open('inv_index.json','r',encoding='utf-8-sig') as f:
    inv_index = json.loads(f.read())
with open('url_info.json','r',encoding='utf-8-sig') as f:
    url_info = json.loads(f.read())

In [6]:
N = len(url_info)
num_words = sum([i[0] for i in url_info.values()])
avgdl = num_words/N

In [7]:
res = search('каникулы на новый год и рождество')

for link, score in res[:10]:
    url = '{} - {}'.format(score, link)
    display(HTML(str(url)))